### 2.Natural Language to SQL

* OpenAI use Cases - completion API, can be used for text / story completion, and code completion

In [59]:
import os
import pandas as pd
import openai
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

In [60]:
# Import and pass api key
with open(r'C:\Users\mhuh22\Desktop\openai_api.txt', 'r') as file:
    api_key = file.readline().strip()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [61]:
df = pd.read_csv(r'C:\Users\mhuh22\Documents\Python\Kaggle\Data\lending_club_loan_two\lending_club_loan_two.csv')
df.head(3)

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,...,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,...,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,...,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"


In [34]:
# Create a sql engine, and push df to sql table in memory
temp_db = create_engine('sqlite:///:memory:',echo=True)

data = df.to_sql(name='Sales',con=temp_db)

2023-03-30 18:31:49,683 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2023-03-30 18:31:49,685 INFO sqlalchemy.engine.base.Engine ()
2023-03-30 18:31:49,687 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2023-03-30 18:31:49,689 INFO sqlalchemy.engine.base.Engine ()
2023-03-30 18:31:49,692 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Sales")
2023-03-30 18:31:49,693 INFO sqlalchemy.engine.base.Engine ()
2023-03-30 18:31:49,699 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	loan_amnt FLOAT, 
	term TEXT, 
	int_rate FLOAT, 
	installment FLOAT, 
	grade TEXT, 
	sub_grade TEXT, 
	emp_title TEXT, 
	emp_length TEXT, 
	home_ownership TEXT, 
	annual_inc FLOAT, 
	verification_status TEXT, 
	issue_d TEXT, 
	loan_status TEXT, 
	purpose TEXT, 
	title TEXT, 
	dti FLOAT, 
	earliest_cr_line TEXT, 
	open_acc FLOAT, 
	pub_rec FLOAT, 
	revol_bal FLOAT, 
	revol_util FLOAT, 


In [44]:
with temp_db.connect() as conn:
    # Makes the connectin
    result = conn.execute(text("SELECT AVG(loan_amnt) FROM Sales"))

2023-03-30 18:39:51,350 INFO sqlalchemy.engine.base.Engine SELECT AVG(loan_amnt) FROM Sales
2023-03-30 18:39:51,352 INFO sqlalchemy.engine.base.Engine ()


In [46]:
result.fetchall()

[(14113.888089286165,)]

In [47]:
def create_table_definition(df):
    prompt = """### sqlite SQL table, with it properties
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))
    
    return prompt

In [51]:
create_table_definition(df)

'### sqlite SQL table, with it properties\n    #\n    # Sales(loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address)\n    #\n    '

In [52]:
def prompt_input():
    nlp_text = input("Enter the info you want: ")
    return nlp_text

In [73]:
prompt_input()

Enter the info you want: average loan amount by term


'average loan amount by term'

In [77]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition + query_init_string

In [86]:
nlp_text = prompt_input()
combine_prompts(df,nlp_text)

Enter the info you want: average loan amount by term


'### sqlite SQL table, with it properties\n    #\n    # Sales(loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address)\n    #\n    ### A query to answer: average loan amount by term\nSELECT'

In [87]:
prompt = combine_prompts(df,nlp_text)

In [88]:
print(prompt)

### sqlite SQL table, with it properties
    #
    # Sales(loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address)
    #
    ### A query to answer: average loan amount by term
SELECT


In [89]:
response = openai.Completion.create(
    model = 'text-davinci-003',
    prompt = combine_prompts(df,nlp_text),
    temperature = 0,
    max_tokens = 150,
    top_p = 1.0,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = ['#',';']
)

In [90]:
response

<OpenAIObject text_completion id=cmpl-6zwvbraVt57eGTKnBTqVeieAqd56p at 0x162a6385c50> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " AVG(loan_amnt)\nFROM Sales\nGROUP BY term"
    }
  ],
  "created": 1680222547,
  "id": "cmpl-6zwvbraVt57eGTKnBTqVeieAqd56p",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 148,
    "total_tokens": 164
  }
}

In [91]:
response['choices'][0]['text']

' AVG(loan_amnt)\nFROM Sales\nGROUP BY term'

In [92]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(" "):
        query = "SELECT" + query
    return query

In [93]:
print(handle_response(response))

SELECT AVG(loan_amnt)
FROM Sales
GROUP BY term
